In [1]:
#import libs
#https://www.kaggle.com/code/asiekkinen/nfl-people-tracking-yolov5?fbclid=IwAR3tnN16HuCWeY0uyiq-XKlr4554ep19qlbuiF71cBlENexa9Tq-TQqiO68
try:
  !pip install ffmpeg-python
  import ffmpeg
  from google.colab import output
  from google.colab import drive
  import os
  import numpy as np
  import pandas as pd
  import matplotlib.pyplot as plt
  from google.colab.patches import cv2_imshow
  from tqdm.auto import tqdm
  from PIL import Image, ImageDraw
  from scipy.spatial import distance_matrix
  import cv2
  import torch
  from IPython.display import Video
  from glob import glob

  print("Importing successfull! ✅")
except:
  print("E: Importing failed! ❌")
#mount drive folder
try:
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/IPCV/PDBR

  print("Drive folder mounted! ✅")
except:
  print("E: Drive mounting failed! ❌")
#Load model
try: 
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
  print("Model loaded! ✅")
except:
  print("E: Model load failed! ❌")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Importing successfull! ✅
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1P7jytiOKZWB3PHpK3ImyLrqM6qDIqLPC/PDBR
Drive folder mounted! ✅


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-12 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Model loaded! ✅


In [3]:
sequences={"badminton":'Videos/badminton',
           "fall":'Videos/fall',
           "overpass":'Videos/overpass',
           "backdoor":'Videos/backdoor',
           "cubicle":'Videos/cubicle',
           "busStation":'Videos/busStation',
           "abandonedBox":'Videos/abandonedBox',
           "copyMachine":'Videos/copyMachine',
           "office":'Videos/office',
           "tud-campus-sequence":'Videos/tud-campus-sequence'}

def get_frames(sequence_name):
  filepath=sequences[sequence_name]
  frame_names=sorted(glob(filepath+"/*.jpg"))
  frames=[]
  
  for i, frame_name in enumerate(frame_names):
    frame=cv2.imread(frame_name)
    frames.append(frame)
  h, w = frames[0].shape[:2]
  
  return frames, h, w

def get_predictions(frames):
  count = 0
  predictions = []
  boxes = []
  model.conf = 0.3  # confidence threshold (0-1)
#  model.iou = 0.5  # NMS IoU threshold
#      agnostic = False  # NMS class-agnostic
#      multi_label = False  # NMS multiple labels per box
#      classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
#      max_det = 1000  # maximum number of detections per image
#      amp = False  # Automatic Mixed Precision (AMP) inference
  
  # model.iou = 0.45  # NMS IoU threshold (0-1)
  for frame in tqdm(frames):
      df = model(frame).pandas().xyxy[0]  # Predict
      df = df[df["class"] == 0]  # Restrict to persons
      df = df.reset_index(drop=True)
      if not df.empty:
        # Calculate some properties
        df[["center_x", "center_y"]] = df.apply(
            lambda row: [row["xmin"] + (row["xmax"] - row["xmin"]) // 2, row["ymin"] + (row["ymax"] - row["ymin"]) // 2], 
            axis=1, 
            result_type="expand"
        )
        # df["area"] = df.apply(lambda row: (row["xmax"] - row["xmin"]) * (row["ymax"] - row["ymin"]), axis=1)
      boxes.append(df)
      predictions.append(df)


      for v in range(len(df["xmin"].values)):
        
        top_left_x = int(df["xmin"].values[v])
        top_left_y = int(df["ymin"].values[v])
        bot_left_x = int(df["xmax"].values[v])
        bot_left_y = int(df["ymax"].values[v])

        cv2.rectangle(frame, (top_left_x, top_left_y), (bot_left_x, bot_left_y), color=(0,255,0), thickness=2)
        
      cv2.imwrite("results/5s/fall/"+ str(count) + ".jpg", frame)
      count +=1
  return predictions, boxes


def draw_bounding_boxes(frames, predictions, color):
    frames_with_bbox = []
    
    # Draw rectangles
    for frame, df in tqdm(zip(frames, predictions)):
        im = Image.fromarray(frame)
        im_draw = ImageDraw.Draw(im)
        for _, row in df.iterrows():  # Iterate persons
            im_draw.rectangle([(row["xmin"], row["ymin"]), (row["xmax"], row["ymax"])], outline=color)
        frames_with_bbox.append(np.array(im))
         
    return frames_with_bbox


In [4]:
name = "fall"

frames,h,w=get_frames(name)
predictions, boxes = get_predictions(frames)


  0%|          | 0/4000 [00:00<?, ?it/s]

In [29]:
# frame_with_boxes = draw_bounding_boxes(frames, predictions,"blue")

0it [00:00, ?it/s]

In [30]:
"""height,width,layers=frame_with_boxes[1].shape

video=cv2.VideoWriter(("results/5s/" +name +".avi"),
cv2.VideoWriter_fourcc(*"MJPG"), 30,(width,height))

for j in range(0,len(frame_with_boxes)):
      video.write(frame_with_boxes[j])

# cv2.destroyAllWindows()
video.release()"""

In [31]:
"""# "./Videos/copyMachine/in003398.jpg"; (369, 130, 55, 153):-0.7472,  (399, 135, 71, 205):-1.3645, (498, 215, 42, 126):-1.3979;
file1 = open(str('Videos/'+ name +'/dataIn.txt'), 'r')
data = []
encoding = 'utf-8'
counter = 0;
for box in boxes:
  line = file1.readline()
  string = "\"./Videos/{}\"; ".format(line[:-1])
  for v in range(len(box["xmin"].values)):
    top_left_x = int(box["xmin"].values[v])
    top_left_y = int(box["ymin"].values[v])
    width = int(box["xmax"].values[v]) - top_left_x
    height = int(box["ymax"].values[v]) - top_left_y

    string += f'({top_left_x}, {top_left_y}, {width}, {height}):{box["confidence"].values[v]},'

  if string[-1] == ',':
    string = string[:-1]
    string += ';'
  
  data.append(string+'\n')

filename = "results/5xliou/" + name + '.idl'
print(filename)
with open(filename, 'wb') as f:
  for i in data:
    bytes_to_write = i.encode(encoding)
    f.write(bytes_to_write)"""

'# "./Videos/copyMachine/in003398.jpg"; (369, 130, 55, 153):-0.7472,  (399, 135, 71, 205):-1.3645, (498, 215, 42, 126):-1.3979;\nfile1 = open(str(\'Videos/\'+ name +\'/dataIn.txt\'), \'r\')\ndata = []\nencoding = \'utf-8\'\ncounter = 0;\nfor box in boxes:\n  line = file1.readline()\n  string = ""./Videos/{}"; ".format(line[:-1])\n  for v in range(len(box["xmin"].values)):\n    top_left_x = int(box["xmin"].values[v])\n    top_left_y = int(box["ymin"].values[v])\n    width = int(box["xmax"].values[v]) - top_left_x\n    height = int(box["ymax"].values[v]) - top_left_y\n\n    string += f\'({top_left_x}, {top_left_y}, {width}, {height}):{box["confidence"].values[v]},\'\n\n  if string[-1] == \',\':\n    string = string[:-1]\n    string += \';\'\n  \n  data.append(string+\'\n\')\n\nfilename = "results/5xliou/" + name + \'.idl\'\nprint(filename)\nwith open(filename, \'wb\') as f:\n  for i in data:\n    bytes_to_write = i.encode(encoding)\n    f.write(bytes_to_write)'

In [5]:
import cv2
import os

image_folder = 'results/5s/fall'
video_name = 'results/5s/fall.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

#video = cv2.VideoWriter(video_name, 0, 1, (width,height))
video=cv2.VideoWriter((video_name),
cv2.VideoWriter_fourcc(*"MJPG"), 30,(width,height))
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

#cv2.destroyAllWindows()
video.release()